In [ ]:
import pandas as pd
import numpy as np

from sklearn.metrics import f1_score, classification_report, roc_curve, roc_auc_score
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

from catboost import CatBoostClassifier

#BASICS

In [ ]:
! pip install catboost --upgrade
!ls 'drive/MyDrive/data/nlp_data' # путь к диску

df = pd.read_csv('drive/MyDrive/data/nlp_data/train.csv') # читаем с диска
df = pd.read_csv('data.csv', encoding='ISO-8859-1') # вид энкодинга

df = pd.Series([1,2,3,4,5,6]).reset_index() # сбросить индекс чтобы из Series сделать DataFrame
df.merge(df2, how='left', on='price') # сджойнить столбцы по ключу price
df.rename({'abc' : 'ABC'}, axis=1) # переименовать стобец

(df['income'] - df['income'].mean()).abs().mean() # вычисляем среднее отклонение
np.sqrt(((df['income'] - df['income'].mean())**2).sum() / (len(df)-1)) # std руками

df.info() # инфо
df.describe() # статистика по числовым значениям
df.sample(5) #вернуть 5 случайных строк
df.isna().mean() # процент пропусков
df['target'].mean() # чем больше значение тем больше 1
df['asd'].unstack() # разбить строки на столбы ( после group by)
df.style.bar() # раскрашывает столбец в процентном соотношении
df[df['asd'] == 'asd'].sort_values('Count', ascending=False) # отсортировать стобец по какому то значению по убыванию
df['asd'] = pd.qcut(df['abc'],5) #сгруппировать данные по 5 категориям
df['asd'] = (df['abc'] >= 0.2) * 1 # выбрать 20% и поставить им 1 если тру и 0 если фолс
df['CustomerID'] = df['CustomerID'].fillna(-999999)
(~df['asd'].isna()) * 1 # ~ заменяет True на False, там где NaN будет 0
df[df['customer_id'].isnull()] # показать только NaN
df['age_bin'] = pd.cut(df['age'],10) # сгруппировать непрерывную переменную на 10 групп

df.groupby('year')['price'].agg(['count', 'mean', 'median']) # группирует годы в индексы цену в значения
t = df.groupby('invoice_month')['revenue'].agg(['count', 'sum']).reset_index()
t = df.groupby(['CustomerID','month'])[['revenue']].agg(['sum']).reset_index() # группировка по клиентам с суммой по месяцу
df.columns = ['_'.join(col).strip() for col in df.columns.values] # убрать мультииндекс

df.columns.str.lower() #показать колонки маленькими буквами

# DATES 

In [ ]:
df['date'] = pd.to_datetime(df['date'],format='%Y-%m-%d %H:%M:%S') # преобразовать дату '2020-02-14 01:27:01'
df['invoice_date'] = pd.to_datetime(df['invoice_date'], format='%m/%d/%Y %H:%M') # преобразовать дату '12/1/2010 8:26'
df['invoice_date'].dt.month.value_counts() # количество по месяцам
df['invoice_month'] = df['invoice_date'].to_numpy().astype('datetime64[M]') # проставить каждой дате 1 число месяца

# создать диапазон дат с мин до макс в 1 первому числу месяца
min_dt = df['invoice_month'].min()
max_dt = df['invoice_month'].max()
pd.date_range(min_dt, max_dt, freq='MS') # MS month start frequency, каждый первый день
pd.date_range('2011-01-01', '2012-01-01', freq='MS') # или так

#FOR ANALYTICS

In [ ]:
import phik
from phik.report import plot_correlation_matrix
from phik import report
phik_matrix = df.phik_matrix()
phik_matrix['asd'].sort_values(ascending=False)

In [ ]:
mean_absolute_error(df['true'], df['pred'])
mean_absolute_percentage_error(df['true'], df['pred'])

#DEFS

In [ ]:
# соединить несколько файлов в один
names_by_year = {}
for year in range(1880, 2018):
    names_by_year[year] = pd.read_csv(
        f'names/yob{year}.txt',
        names=['Name','Gender','Count']
    )

names_all = pd.concat(names_by_year).reset_index(level=1, drop=True)
names_all

In [ ]:
def device_group(x):
  if 'iphone' in x:
    return 'iphone'
  if 'huawei' in x:
    return 'huawei'
  if 'samsung' in x:
    return 'samsung'
  if 'tecno' in x:
    return 'tecno'
  if 'xiaomi' in x:
    return 'xiaomi'
  if 'nokia' in x:
    return 'nokia'
  return 'misc'

  df['device_group'] = df['device_name'].str.lower().apply(device_group)

#TRAIN_TEST

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, 
                                                    random_state=42)
train = df.sample(frac=0.8).copy() # берём 80% от дф
val = df[~df.index.isin(train.index)].copy() # берем всё что не вошло в 80%

In [ ]:
model = CatBoostClassifier()
model.fit(train[X_col], train[y_col], eval_set=(val[X_col], val[y_col]))
model.predict(val[X_col])

In [ ]:
f1_score(t['y_true'], t['y_pred'])
print(classification_report(t['y_true'], t['y_pred']))

In [ ]:
t = pd.DataFrame({'y_true': val['target'].copy(), 
                  'y_pred': model.predict(val[X_col]),
                 'y_score': model.predict_proba(val[X_col])[:,1]})

#PLOTS

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('dark_background')

In [ ]:
from pylab import rcParams # изменить размер графиков
rcParams['figure.figsize'] = 12,9

In [ ]:
df.groupby('year')['price'].median().plot() # график медианной зависимости цены от года
plt.barh(df2['Name'], df2['Count']) #горизонтальный столбцы по двум стобцам